# <font color='blue'>Projeto com Grafos - Transferências de Jogadores</font>

## <font color='blue'>Transferência de Jogadores em grafos - Estudo</font>


In [23]:
#Importando Pacotes
import pandas as pd
import sqlite3
import numpy as np
!pip -q install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
!pip -q install pyvis
from pyvis.network import Network
net = Network()
import networkx as nx
import re


##Pre sets##

#Maximo Colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [24]:
# Extraindo as tabelas do .sqlite
con = sqlite3.connect("Arquivo - Soccer/database.sqlite")
players = pd.read_sql_query("SELECT * FROM Player", con)
players_stats = pd.read_sql_query("SELECT * FROM Player_Attributes", con)
teams = pd.read_sql_query("SELECT * FROM Team", con)
league = pd.read_sql_query("SELECT * FROM League", con)
Matches = pd.read_sql_query("SELECT * FROM Match", con)

con.close()

In [25]:
#Substituir caracteres e espaço por "_" no dataframe de Times

teams.team_long_name.replace(" ","_", regex=True, inplace=True)
teams.team_long_name.replace("-","_", regex=True, inplace=True)
teams.team_long_name.replace("\\.","_", regex=True, inplace=True)
teams.team_long_name.replace("1__.","_", regex=True, inplace=True)

In [26]:
#Merge de Matches, League e Teams

CountryClub = pd.merge_asof(Matches, league, on="country_id", by='country_id')
CountryClub = CountryClub.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

#Separar Times pelas Ligas

LeagueTeam = CountryClub[['name','team_long_name']].drop_duplicates()

In [27]:
#Merge entre Jogadores e Times

#Transformar as colunas de jogadores em uma coluna unica
tmp = Matches.melt(id_vars=['season','home_team_api_id'], 
               value_vars=['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5','home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11'],
                value_name='player_api_id')

#Excluir os valores nulos
tmp = tmp.dropna()

#Transformar os valores da planilha players em float
players = players.astype({"player_api_id": float,})

#Realizar o merge entre Matches e Players
left_sorted = tmp.sort_values(by="player_api_id")
right_sorted = players.sort_values(by="player_api_id")

tmp = pd.merge_asof(left_sorted, right_sorted, on="player_api_id")

#Realizar o merge entre Jogadores e Times
TeamsPlayer = tmp.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

#Deixar em classificação alfabetica
TeamsPlayer = TeamsPlayer.sort_values('id_x')

#salvar em csv

TeamsPlayer.to_csv(r'TeamsPlayer.csv',header=True, index=True, sep=',')



In [28]:
TeamsPlayer = pd.read_csv(r'TeamsPlayer.csv')

#Criando cesta de dados

basket = (TeamsPlayer
          .groupby(['player_name', 'team_long_name'])['id_x']
          .sum().unstack().reset_index().fillna(0)
          .set_index('player_name'))

#Transformando informações em binário para utilização do apriori
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

#Criando dataset de frequencia
frequent_itemsets = apriori(basket_sets, min_support=0.0001, use_colnames=True, max_len=2, verbose=True)

#Criando associação dos dados da cesta
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

#salvar em csv
rules.to_csv(r'rules.csv',header=True, index=True, sep=',')

Processing 87320 combinations | Sampling itemset size 2


In [29]:
#Ler arquivo CSV da cesta de dados
rules = pd.read_csv(r'rules.csv')

#Limpar dados do CSV
rules["antecedents"] = rules["antecedents"].str.replace(r'frozenset\(|\)|\{|\}\|\'|\'|\}',"")
rules["consequents"] = rules["consequents"].str.replace(r'frozenset\(|\)|\{|\}\|\'|\'|\}',"")

In [30]:
#Realizar Merge entre a Cesta com a liga de Times
rules_leagues = rules.merge(LeagueTeam, how='left', left_on="antecedents", right_on="team_long_name")
rules_leagues = rules_leagues.merge(LeagueTeam, how='left', left_on="consequents", right_on="team_long_name")

#salvar em csv
rules_leagues.to_csv(r'rules_leagues.csv',header=True, index=True, sep=',')

In [31]:
rules_leagues.head()

,Unnamed: 0,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,name_x,team_long_name_x,name_y,team_long_name_y
0,0,AC_Ajaccio,AJ_Auxerre,0.004252,0.003769,0.000387,0.090909,24.118881,0.000371,1.095854,France Ligue 1,AC_Ajaccio,France Ligue 1,AJ_Auxerre
1,1,AJ_Auxerre,AC_Ajaccio,0.003769,0.004252,0.000387,0.102564,24.118881,0.000371,1.109547,France Ligue 1,AJ_Auxerre,France Ligue 1,AC_Ajaccio
2,2,FC_Lorient,AC_Ajaccio,0.008891,0.004252,0.000193,0.021739,5.112154,0.000155,1.017875,France Ligue 1,FC_Lorient,France Ligue 1,AC_Ajaccio
3,3,AC_Ajaccio,FC_Lorient,0.004252,0.008891,0.000193,0.045455,5.112154,0.000155,1.038304,France Ligue 1,AC_Ajaccio,France Ligue 1,FC_Lorient
4,4,AC_Ajaccio,FC_Nantes,0.004252,0.006089,0.000193,0.045455,7.465368,0.000167,1.041240,France Ligue 1,AC_Ajaccio,France Ligue 1,FC_Nantes


In [32]:
CoresLigas = {'France Ligue 1':'#0072bb',
              'Switzerland Super League':'#d52b1e',
              'Netherlands Eredivisie':'#FF4F00',
              'Belgium Jupiler League':'#FFDDA24',
              'Spain LIGA BBVA':'#f1bf00',
              'Italy Serie A':'#008c45',
              'England Premier League':'#FFFFFF',
              'Portugal Liga ZON Sagres':'#FF0000',
              'Scotland Premier League':'#0065BF',
              'Germany 1. Bundesliga':'#000000',
              'Poland Ekstraklasa':'#dd0000'}

In [76]:
got_net = Network(height='100%', width='100%')

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv('rules_leagues.csv')

time1 = got_data['antecedents']
time2 = got_data['consequents']
trocas = got_data['support']
liga1 = got_data['name_x'].replace(list(CoresLigas.keys()),list(CoresLigas.values()))
liga2 = got_data['name_y'].replace(list(CoresLigas.keys()),list(CoresLigas.values()))



edge_data = zip(time1, time2, trocas, liga1,liga2)

for e in edge_data:
    time1 = e[0]
    time2 = e[1]
    trocas = e[2]
    liga1 = e[3]
    liga2 = e[4]

    got_net.add_node(time1, time1, title=time1, color=liga1, shape='box')    
    got_net.add_node(time2, time2, title=time2, color=liga2, shape='box')    
    got_net.add_edge(time1, time2, value=trocas)
    

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
    #node['color'] = liga
    
got_net.set_edge_smooth('cubicBezier')
    
#got_net.show_buttons(filter_=['interaction'])
got_net.show('transacoesdetimes.html')